#### This notebook reads ``uk_addresses.csv`` and calls ``api.openalpr.com`` to get car metdata like model, colour, type etc. The output file it creates is a JSON file that then gets ingested into Postgresql Database

In [54]:
#!/usr/bin/python

import requests
import base64
import json
import uuid
#pip3 install Faker
from faker import Faker
import random
import pandas
from geopy.geocoders import Nominatim

def seed_db_creator(file_name):
  IMAGE_PATH = '/Users/karasing/git/jumpstart-library/pattern2-smart-city/source/dataset/images/'+str(file_name)+'.png'
  SECRET_KEY = 'sk_90c281aebbfc04d52d6c2e86'
  results = {}
  with open(IMAGE_PATH, 'rb') as image_file:
      img_base64 = base64.b64encode(image_file.read())
      ## Calling api.openalpr.com to get car metdata like model, colour, type etc
  url = 'https://api.openalpr.com/v3/recognize_bytes?recognize_vehicle=1&country=us&secret_key=%s' % (SECRET_KEY)

  '''
  - openalpr.com API credits (using free account) are limited so for testing we should  leverage alpr_api_response.json file , instead of calling the API
  - if you want to get input data from  alpr_api_response.json instead of openalpr.com API, then
      - Uncomment : dump_file and data variables
      - Comment : r and data variable
  '''
  #dump_file = open("alpr_api_response.json")
  #data = json.load(dump_file)
  r = requests.post(url, data = img_base64)
  data = json.loads( json.dumps(r.json(), indent=2))
  
 ## Read UK Addresses CSV File that contains 50 addresses
  df = pandas.read_csv("uk_addresses.csv")
  #address_file = open("addresses.json")
  #address_data = json.load(address_file)
  
  for i in data["results"]:
    customer_id = str(uuid.uuid4())
    fake_owner_name = fake.name()
    random_int = random.randint(0,49)

    #print(fake_address_details)
    #fake_lat_long = fake.local_latlng()

    ##### Vehicle Information #####
    results.update({"vehicle_registered_plate_number": i['plate']})
    results.update({"vehicle_color": i["vehicle"]["color"][0]["name"]})
    results.update({"vehicle_make": i["vehicle"]["make"][0]["name"]})
    results.update({"vehicle_body_type": i["vehicle"]["body_type"][0]["name"]})
    results.update({"vehicle_make_model": i["vehicle"]["make_model"][0]["name"]})
    results.update({"vehicle_model_year": i["vehicle"]["year"][0]["name"]})
    results.update({"vehicle_registered_city": df.loc[random_int].at["City"]})
   ###### Owner Information #####
    results.update({"vehicle_owner_name": fake_owner_name})
    results.update({"vehicle_owner_address": df.loc[random_int].at["Address"]})
    results.update({"vehicle_owner_city": df.loc[random_int].at["City"]})
    results.update({"vehicle_owner_zip_code": df.loc[random_int].at["Postcode"]})
    results.update({"vehicle_owner_contact_number": df.loc[random_int].at["Phone number"]})
   ###### Customer Information #####
    results.update({"customer_id": customer_id})
    results.update({"customer_balance": random.randint(100,999)})
    results.update({"customer_name": fake_owner_name})
    results.update({"customer_address": df.loc[random_int].at["Address"]})
    results.update({"customer_city": df.loc[random_int].at["City"]})
    results.update({"customer_zip_code": df.loc[random_int].at["Postcode"]})
    results.update({"customer_contact_number": df.loc[random_int].at["Phone number"]})
    results.update({"file_name": str(file_name)+'.png'})
    if not os.path.isfile(db_fname):
      file.append(results)
      with open(db_fname, mode='w') as f:
        f.write(json.dumps(file, indent=2))
    else:
      with open(db_fname) as feedsjson:
          feeds = json.load(feedsjson)
      feeds.append(results)
      with open(db_fname, mode='w') as f:
       f.write(json.dumps(feeds, indent=2))

fake = Faker( 'en_GB')
file = []
db_fname = "database.json"
## Set delete_file to "false" if you do not want to delete existing database.json file in the PWD
delete_file = False
if delete_file:
  if os.path.isfile(db_fname):
    print("Deleting "+db_fname+" file")
    os.remove(db_fname)
else:
  print("Skipping Delete")

for image_file_name in range(11,12):
  print("Processing image "+str(image_file_name))
  seed_db_creator(image_file_name)


Skipping Delete
Processing image 11


### Reading resultant json file and ingesting the data to Postgresql Database

In [9]:
! pip3 install sqlalchemy psycopg2

You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [39]:
from sqlalchemy import create_engine, Column, Integer, String, Sequence
from sqlalchemy.ext.declarative import declarative_base
import json, urllib.request

## JSON Data URL
JSON_DB_URL = "database.json"

## Database details and connection
DB_USER = os.getenv('DB_USER', 'dbadmin')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'HT@1202k')
DB_HOST = os.getenv('DB_HOST', '127.0.0.1')
DB_NAME = os.getenv('DB_NAME','pgdb')
TABLE_NAME = os.getenv('TABLE_NAME','vehicle_metadata')

engine = create_engine('postgresql://'+DB_USER+':'+DB_PASSWORD+'@'+DB_HOST+'/'+DB_NAME, connect_args={})
Base = declarative_base()
TABLE_ID = Sequence(TABLE_NAME+'_id_seq', start=1000)

class Create_Table(Base):
    __tablename__ = TABLE_NAME
    id = Column(Integer, TABLE_ID,  primary_key=True, index=True, server_default=TABLE_ID.next_value())
    vehicle_registered_plate_number = Column(String, primary_key=True, index=True, unique=True)
    vehicle_color = Column(String)
    vehicle_make = Column(String)
    vehicle_body_type = Column(String)
    vehicle_make_model = Column(String)
    vehicle_model_year = Column(String)
    vehicle_registered_city = Column(String)
    vehicle_owner_name = Column(String)
    vehicle_owner_address = Column(String)
    vehicle_owner_city = Column(String)
    vehicle_owner_zip_code = Column(String)
    vehicle_owner_contact_number = Column(String)
    customer_id = Column(String)
    customer_balance = Column(Integer)
    customer_name = Column(String)
    customer_address = Column(String)
    customer_city = Column(String)
    customer_zip_code = Column(String)
    customer_contact_number = Column(String)

## Create Table if does not exists
Create_Table.__table__.create(bind=engine, checkfirst=True)

with urllib.request.urlopen(JSON_DB_URL) as url:
    data = json.loads(url.read().decode())

connection = engine.connect()

for count in range(len(data)):
    try:
        connection.execute(f"""INSERT INTO public.{TABLE_NAME}(vehicle_registered_plate_number,vehicle_color,vehicle_make,vehicle_body_type,vehicle_make_model,vehicle_model_year,vehicle_registered_city,vehicle_owner_name,vehicle_owner_address,vehicle_owner_city,vehicle_owner_zip_code,vehicle_owner_contact_number,customer_id,customer_balance,customer_name,customer_address,customer_city,customer_zip_code,customer_contact_number) 
VALUES('{data[count]['vehicle_registered_plate_number']}', '{data[count]['vehicle_color']}', '{data[count]['vehicle_make']}', '{data[count]['vehicle_body_type']}', '{data[count]['vehicle_make_model']}', '{data[count]['vehicle_model_year']}', '{data[count]['vehicle_registered_city']}', '{data[count]['vehicle_owner_name']}', '{data[count]['vehicle_owner_address']}', '{data[count]['vehicle_owner_city']}', '{data[count]['vehicle_owner_zip_code']}', '{data[count]['vehicle_owner_contact_number']}', '{data[count]['customer_id']}', '{data[count]['customer_balance']}', '{data[count]['customer_name']}', '{data[count]['customer_address']}', '{data[count]['customer_city']}', '{data[count]['customer_zip_code']}', '{data[count]['customer_contact_number']}')""")
    except:
        print("====================== Error inserting record to database ======================")
        print(sys.exc_info()[1])
    else:
        print(" Record added to DB successfully")


OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused
	Is the server running on host "127.0.0.1" and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/14/e3q8)